# jupyter_utils Module

Collection of utlity methods to be used in Jupyter notebook contexts

- HTML display
- Markdown display
- Utils for use in colab
- 



## HTML Display

Collection of utilities to display HTML

TODO as needed
 - SVG
 - IFrames 

In [2]:
from IPython.display import display, HTML, Markdown

In [32]:
from enum import StrEnum
class TextAlign (StrEnum):
    DEFAULT = "justify"
    RIGHT = "right"
    LEFT = "left"

class DisplayHTML:
    """
    Collection of jupyter visualization methods
    """
    @staticmethod
    def color_box(txt, title=None, bg="pink", fg="black", align= TextAlign.DEFAULT):
        if title is not None:
            txt = f"<b>{title}</b><br><hr><br>{txt}"

        MARGIN_AMOUNT="100px"
        match align:
            case TextAlign.LEFT:        
                margin = f"margin-right:{MARGIN_AMOUNT};"
            case TextAlign.RIGHT:
                margin = f"margin-left:{MARGIN_AMOUNT};"
            case _:
                margin = ""

        display(HTML(f"<div style='border-radius:15px;padding:15px;{margin}text-align:{str(align)};background-color:{bg};color:{fg};'>{txt}</div>"))    
    
    @staticmethod
    def text(txt, bg=None, fg=None, align=None):
         bg = f"background-color:{bg}" if bg else ''
         fg = f"color:{fg}" if fg else ''
         align = f"text-align:{align}" if align else ''
         display(HTML(f"<p style='{bg};{fg};{align};'>{txt}</p>"))

In [34]:
DisplayHTML.color_box("This has no title")
DisplayHTML.color_box("This is with a title", title="My Title")
DisplayHTML.color_box("black on green bg and justified right", fg="red", bg="lightgreen", align=TextAlign.RIGHT)
DisplayHTML.color_box("red on blue bg and justified left", fg="red", bg="blue", align=TextAlign.LEFT)

DisplayHTML.text("Plain red text, default alignment", fg="red")
DisplayHTML.text("Plain yellow text, aligned Right", fg="yellow", align="right")


## Markdown display

### API

In [9]:
from IPython.display import display, Markdown
import json

class Md:
     @staticmethod
     def h(title:str, title_level:int|None=None):
          """
          Heading. 
          `title_level` defaults to 1
          """
          title_level = title_level if title_level else 1
          title_hashtrain = eval(f"\"#\"*{title_level}") if title_level > 1 else "#"
          Md.md(f"{title_hashtrain} {title}")

     @staticmethod
     def json(jsn, indent=4):
          """
          Display JSON.
          If String      → displayed as is
          If Json object → Displayed with the optional indent
                           Optional indent defaults to 4
          """          
          Md.md(
               Md.json_fmt(jsn, indent)
          )
     
     @staticmethod
     def hr():
          """
          like the <hr> of HTML
          Draws a separator using the "----" Md
          """
          Md.md("----")

     @staticmethod            
     def code(code_block_str:str, code_lang:str|None=None):
          """
          Display Markdown code
          code_lang defaults to empty which will simply produce a ```...``` block
          """          
          Md.md(
               Md.code_fmt(code_block_str, code_lang)
          )

     @staticmethod
     def md(markdown_text:str):
          """
          Display markdown formatted text
          """
          display(Markdown(markdown_text))

     @staticmethod
     def code_fmt(code_block:str, code_lang:str|None=""):
          return f"```{code_lang}\n{code_block}\n```"
     
     @staticmethod
     def json_fmt(jsn, indent=4):
          if not isinstance(jsn, str):               
               jsn = json.dumps(jsn, indent=indent)               

          return Md.code_fmt(jsn, "json")
     

In [11]:
Md.h("This is a title. Defaults to H1")

Md.hr()
Md.h("This is a title at level 3", title_level=3)

Md.hr()

Md.h("This displays a json block at default indent of 4")
Md.json(
    {
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia"
                }
            },
            "required": [
                "location"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
})

# This is a title. Defaults to H1

----

### This is a title at level 3

----

# This displays a json block at default indent of 4

```json
{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogot\u00e1, Colombia"
                }
            },
            "required": [
                "location"
            ],
            "additionalProperties": false
        },
        "strict": true
    }
}
```


### Improve callability - Eliminate `Md.`

I have the following code to display test output

```python
Md.h("get_weather tool schema output", title_level=3)
Md.json(tool_schema)
Md.hr()
Md.h("get_weather in-prompt schema output", title_level=3)
Md.code(Tool.build_inprompt_tool_schema(tool_schema))

Md.h("Deserialized from string", title_level=3)
Md.code(arg_deserializer(SERIALIZED_DATA_SAMPLE))

Md.h("Function called with deserialized", title_level=3)
Md.md("👉" + str(fn_caller(arg_deserializer(SERIALIZED_DATA_SAMPLE))))
```

Unlike some language, I cannot simply do a `from Md import *` and use `h(..), hr()..` etc to reduce clutter. 
 - Either put these methods into global scope of a module and import from there _(the only reason they are under a class is so that can all go into the one jupyter_utils.py module)_
 - Convert to a builder. This might anyway have to be done. Visually, jupyter might render things differently in the two case of _building entire Markdown and then displaying it_ vs _displaying each piece in a separate call_.

 Does this look better:
 
```python
h("get_weather tool schema output", title_level=3)
json(tool_schema)
hr()
h("get_weather in-prompt schema output", title_level=3)
code(Tool.build_inprompt_tool_schema(tool_schema))
h("Deserialized from string", title_level=3)
code(arg_deserializer(SERIALIZED_DATA_SAMPLE))
h("Function called with deserialized", title_level=3)
md("👉" + str(fn_caller(arg_deserializer(SERIALIZED_DATA_SAMPLE))))
```

Nah..

What if I use plain strings and use an Md formatter instead ?? (using `---` instead of triple quotes as nesting them was confusing Md)

```python
Md.md(f"""
### get_weather tool schema output
{Md.json_fmt(tool_schema)}
----
### get_weather in-prompt schema output
{Md.code_fmt(Tool.build_inprompt_tool_schema(tool_schema))}

### Deserialized from string
{Md.code_fmt(arg_deserializer(SERIALIZED_DATA_SAMPLE))}

# Function called with deserialized

👉 {str(fn_caller(arg_deserializer(SERIALIZED_DATA_SAMPLE))))}
""")
```

Honestly, looks better to me. More flexible and way more quickly customizable.

# Developments/Experiments

## HTML display

See what it'll take to extend `color_box` to support a two-actor conversation. This will be the most common case involving `role=user` and `role=assistant` in a chat application. 

### Specify fg and bg colors

In [25]:

def color_box(txt, title=None, bg="pink", fg="black"):
    if title is not None:
        txt = f"<b>{title}</b><br><hr><br>{txt}"

    display(HTML(f"<div style='border-radius:15px;padding:15px;background-color:{bg};color:{fg};'>{txt}</div>"))

color_box("black text on pink")
color_box("black text on yellow", bg="yellow")
color_box("red text on blue", bg="blue", fg="red")

### Specify text alignment - Distinguishes two modes

Alright! Pretty good. However, multi-turn like a phone msg app will be nicely if I can align text to differnt sides of the screen. With two actors, two sides will do. Research leads to a `text-align` property that takes "left", "right" or "justify". Plan text will work but maybe nicer to use an enum.

In [29]:
from enum import StrEnum
class TextAlign (StrEnum):
    DEFAULT = "justify"
    RIGHT = "right"
    LEFT = "left"

# print(str(TextAlign.DEFAULT))

def color_box(txt, title=None, bg="pink", fg="black", align= TextAlign.DEFAULT):
        if title is not None:
            txt = f"<b>{title}</b><br><hr><br>{txt}"

        display(HTML(f"<div style='border-radius:15px;padding:15px;text-align:{str(align)};background-color:{bg};color:{fg};'>{txt}</div>"))    

color_box("black text on pink. Default Aligned")
color_box("black text on yellow. Left aligned", bg="yellow", align = TextAlign.LEFT)
color_box("red text on blue. Right aligned", bg="blue", fg="red", align = TextAlign.RIGHT)

That worked great. One more thing though, with long text blocks, this won't really help (after seeing how it is handled in two turns convos in the `Py_mod_openai_utils_devel.ipynb`). See if we can use sizing of the color-box to help. 


### Shorten box to emphasize alignment with long text that would fill the box.

Idea would be provide space outside the box
 - Space to the left for right-aligned text
 - Space to the right for left-aligned text
 - No space for default.

In [31]:
# Margin is for space outrside the box
# Padding is for space inside the box.
def color_box(txt, title=None, bg="pink", fg="black", align= TextAlign.DEFAULT):
        if title is not None:
            txt = f"<b>{title}</b><br><hr><br>{txt}"

        MARGIN_AMOUNT="100px"
        match align:
            case TextAlign.LEFT:        
                margin = f"margin-right:{MARGIN_AMOUNT};"
            case TextAlign.RIGHT:
                margin = f"margin-left:{MARGIN_AMOUNT};"
            case _:
                margin = ""

        display(HTML(f"<div style='border-radius:15px;padding:15px;{margin}text-align:{str(align)};background-color:{bg};color:{fg};'>{txt}</div>"))    

color_box("black text on pink. Default Aligned")
color_box("black text on yellow. Left aligned", bg="yellow", align = TextAlign.LEFT)
color_box("red text on blue. Right aligned", bg="blue", fg="red", align = TextAlign.RIGHT)